In [ ]:
# Keep only Surface, Molecule, Hads and Error columns in expt_hads_df
hads_nu13_comparison_df = expt_hads_df[['Surface','Molecule','Hads','Error']]
# Remove the 'Physisorbed CO2' row
hads_nu13_comparison_df = hads_nu13_comparison_df[hads_nu13_comparison_df['Molecule'] != 'Physisorbed CO2']
# Rename the columns to Expt. Hads and Expt. Error
hads_nu13_comparison_df.columns = ['Surface','Molecule','Expt. Hads','Expt. Error']

# Save autoSKZCAM and experiment hads into a nested dictionary with surface -> molecule -> experiment or autosKZCAM for plotting in next cell
hads_nu13_comparison_dict = {}

for idx, row in hads_nu13_comparison_df.iterrows():
    surface = row["Surface"]
    molecule = row["Molecule"]
    
    if surface not in hads_nu13_comparison_dict:
        hads_nu13_comparison_dict[surface] = {}
        
    hads_nu13_comparison_dict[surface][molecule] = {
        'Experiment': [int(row['Expt. Hads']), int(row['Expt. Error'])],
        'autoSKZCAM': [0, 0]
    }

# Add the AutoSKZCAM Hads and Error columns to the DataFrame
# Loop over the DataFrame and add the AutoSKZCAM Hads and Error values
autoskzcam_ground_state_hads = []
autoskzcam_ground_state_hads_error = []
system_label_list = []
hads_delta_min_list = []

for idx, row in hads_nu13_comparison_df.iterrows():
    surface = row['Surface']
    molecule = row['Molecule']

    system_expt_hads = int(row['Expt. Hads'])
    system_expt_error = int(row['Expt. Error'])
    system_expt_hads_lower_bound = system_expt_hads - system_expt_error
    system_expt_hads_upper_bound = system_expt_hads + system_expt_error
    
    if len(molecule.split()) == 2:
        if molecule.split()[0] == 'Cluster':
            cluster_size = 'Tetramer' if molecule.split()[1] in ['H2O','CH3OH'] else 'Dimer'
            molecule_label = molecule.split()[1] + ' ' + cluster_size
        else:
            molecule_label = molecule.split()[1] + ' ' + molecule.split()[0]
    elif surface == 'MgO' and molecule == 'H2O':
        molecule_label = 'H2O Monomer'
    elif surface == 'MgO' and molecule == 'N2O':
        molecule_label = 'N2O Parallel'
    elif surface == 'r-TiO2' and molecule == 'CO2':
        molecule_label = 'CO2 Tilted'

    else:
        molecule_label = molecule

    system_label = f'{convert_to_nice_labels["adsorbates"][surface][molecule_label]} on {convert_to_nice_labels["surface"][surface]}'

    system_label_list += [system_label]
    autoskzcam_ground_state_hads += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0]))]
    autoskzcam_ground_state_hads_error += [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]))]
    hads_nu13_comparison_dict[surface][molecule]['autoSKZCAM'] = [int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0])), int(round(autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]) )]

    system_autoskzcam_hads_lower_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] - autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    system_autoskzcam_hads_upper_bound = autoskzcam_final_hads_dict[surface][molecule_label]['Final'][0] + autoskzcam_final_hads_dict[surface][molecule_label]['Final'][1]

    # Find the minimum difference between the experimental Hads and the AutoSKZCAM Hads errors
    if max(system_expt_hads_lower_bound, system_autoskzcam_hads_lower_bound) <= min(system_expt_hads_upper_bound, system_autoskzcam_hads_upper_bound):
        delta_min = 0
    else:
        delta_min = min(abs(system_expt_hads_upper_bound - system_autoskzcam_hads_lower_bound), abs(system_expt_hads_lower_bound - system_autoskzcam_hads_upper_bound))


    hads_delta_min_list += [int(round(delta_min))]


# Add the system labels, AutoSKZCAM Hads and Error columns to the DataFrame
hads_nu13_comparison_df['autoSKZCAM Hads'] = autoskzcam_ground_state_hads
hads_nu13_comparison_df['autoSKZCAM Error'] = autoskzcam_ground_state_hads_error
hads_nu13_comparison_df['DeltaMin'] = hads_delta_min_list
hads_nu13_comparison_df['System'] = system_label_list

# Rearrange the columns
hads_nu13_comparison_df = hads_nu13_comparison_df[['System','Expt. Hads','Expt. Error','autoSKZCAM Hads','autoSKZCAM Error','DeltaMin']]

# Round to nearest integer
hads_nu13_comparison_df = hads_nu13_comparison_df.map(lambda x: int(round(x)) if isinstance(x, float) else x)
hads_nu13_comparison_df.columns.name = 'System' 


hads_nu13_comparison_dict['MgO']['CO2'] = hads_nu13_comparison_dict['MgO']['Chemisorbed CO2']
del hads_nu13_comparison_dict['MgO']['Chemisorbed CO2']

display(hads_nu13_comparison_df)

In [1]:
import numpy as np
from ase import units
from scipy.stats import linregress
from datetime import datetime
from ase.units import mol, kcal, kJ, Hartree, Bohr
import pandas as pd
from ase import io
import re


# Some basic conversion factors
cm1_to_eV = 1 / 8065.54429
hundredcm1 = 100 * cm1_to_eV * 1000
kcalmol_to_meV = kcal / mol * 1000
kjmol_to_meV = kJ / mol * 1000
mha_to_meV = Hartree

In [2]:
def get_mrcc_walltime(filename):
    """
    Reads the walltime from the mrcc.out file.
        
    Parameters
    ----------
    filename : str
        The location of the 'mrcc.out' file to read from.
   
   Returns
    -------
    float
        The walltime in seconds.
        
    Notes
    -----
    This function reads the 'mrcc.out' file and extracts the start and end time information to calculate the total walltime taken by the MRCC calculation. The function calculates the time duration between the timestamps found in the file and returns it in seconds.
    """

    # Get the start and end time from the file
    orig_time = datetime(1, 1, 1, 0, 0)
    total_time = datetime(1, 1, 1, 0, 0)

    f = open(filename)
    a = f.readlines()
    b1 = datetime.strptime(
        a[19].split()[1] + "-" + a[19].split()[2], "%Y-%m-%d-%H:%M:%S"
    )
    b2 = datetime.strptime(
        a[-3].split()[1] + "-" + a[-3].split()[2], "%Y-%m-%d-%H:%M:%S"
    )
    total_time = total_time + (b2 - b1)
    f.close()
    return (total_time - orig_time).total_seconds()

In [7]:
def get_orca_walltime(filename):
    """
    Reads the walltime from the mrcc.out file.
        
    Parameters
    ----------
    filename : str
        The location of the 'mrcc.out' file to read from.
   
   Returns
    -------
    float
        The walltime in seconds.
        
    Notes
    -----
    This function reads the 'orca.out' file and extracts the start and end time information to calculate the total walltime in seconds taken by the ORCA calculation. The function calculates the time duration between the timestamps found in the file and returns it in seconds.
    """

    # Get the start and end time from the file
    orig_time = datetime(1, 1, 1, 0, 0)
    total_time = datetime(1, 1, 1, 0, 0)

    f = open(filename)
    line = f.readlines()[-1].split()
    f.close()

    total_time_minutes = float(line[3])*24*60*60+ float(line[5])*60*60 + float(line[7])*60 + float(line[9])
    return total_time_minutes

In [12]:
get_orca_walltime('Data/05b-Eint_SKZCAM/r-TiO2/H2O/Cluster_7/Molecule-Surface/MP2_aVDZ.orca.out')

166033.0

In [13]:
1*24*60*60+22*60*60+7*60+13

166033

In [11]:
2*60*60+47*60+28

10048

In [3]:
line = 'TOTAL RUN TIME: 0 days 0 hours 0 minutes 12 seconds 900 msec'

In [5]:
float(line.split()[3])*24*60+ float(line.split()[5])*60 + float(line.split()[7]) + float(line.split()[9])

['TOTAL',
 'RUN',
 'TIME:',
 '0',
 'days',
 '0',
 'hours',
 '0',
 'minutes',
 '12',
 'seconds',
 '900',
 'msec']

In [ ]:
fig, axs = plt.subplots(figsize=(3.365,3.5),dpi=600, sharey=True, constrained_layout=True) #,gridspec_kw={'width_ratios':[0.65,2]})

autoskzcam_n2o_parallel_hads = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][0]
autoskzcam_n2o_parallel_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Parallel']['Final'][1]
dft_xc_n2o_parallel_hads = [dft_eads_true_dict['MgO']['N2O Parallel'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_parallel_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Parallel']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]
autoskzcam_n2o_tilted_hads = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][0]
autoskzcam_n2o_tilted_hads_error = autoskzcam_final_hads_dict['MgO']['N2O Tilted']['Final'][1]
dft_xc_n2o_tilted_hads = [dft_eads_true_dict['MgO']['N2O Tilted'][xc_func] + autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][0] for xc_func in crystal_xc_func_ensemble['MgO']]
dft_xc_n2o_tilted_hads_error = [autoskzcam_final_hads_dict['MgO']['N2O Tilted']['DFT DeltaH'][1] for xc_func in crystal_xc_func_ensemble['MgO']]

rutile_n2o_expt_hads = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Hads'].values[0])
rutile_n2o_expt_error = float(expt_hads_df.loc[(expt_hads_df['Surface'] == 'MgO') & (expt_hads_df['Molecule'] == 'N2O'), 'Error'].values[0])

axs.errorbar(list(range(7)), [autoskzcam_n2o_parallel_hads] + dft_xc_n2o_parallel_hads, yerr = [autoskzcam_n2o_parallel_hads_error] + dft_xc_n2o_parallel_hads_error, color=color_dict['red'], alpha=0.7, ls='--', marker='o', capsize=2, label='Parallel')
axs.errorbar(list(range(7)), [autoskzcam_n2o_tilted_hads] + dft_xc_n2o_tilted_hads, yerr = [autoskzcam_n2o_tilted_hads_error] + dft_xc_n2o_tilted_hads_error, color=color_dict['blue'], alpha=0.7, ls='--', marker='o', capsize=2, label='Tilted')

axs.plot([-1,9], [rutile_n2o_expt_hads, rutile_n2o_expt_hads], color='k', linestyle='--', label=r'Expt. $H_\textrm{ads}$')
axs.fill_between([-1,9], rutile_n2o_expt_hads - rutile_n2o_expt_error, rutile_n2o_expt_hads + rutile_n2o_expt_error, color=color_dict['black'],alpha=0.1, edgecolor='none')


axs.set_xticks(list(range(7)))
axs.set_xticklabels([r'\textbf{autoSKZCAM}'] + [convert_to_nice_labels['xc_functionals'][xc_func] for xc_func in crystal_xc_func_ensemble['MgO']],rotation=90,ha='center')
axs.legend(frameon=False, fontsize=8)
axs.set_ylabel(r'$H_\textrm{ads}$ [meV]')
axs.set_ylim([-600,-300])
axs.set_xlim([-0.5,6.5])

plt.savefig('Figures/SI_Figure-Hads_rutile_N2O_Configurations.png')